### Load tensorflow

In [1]:
import tensorflow as tf

In [2]:
#Reset Default graph - Needed only for Jupyter notebook
tf.reset_default_graph()

### Collect Data

In [3]:
import pandas as pd

In [4]:
data = pd.read_csv('./prices.csv')

### Check all columns in the dataset

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 851264 entries, 0 to 851263
Data columns (total 7 columns):
date      851264 non-null object
symbol    851264 non-null object
open      851264 non-null float64
close     851264 non-null float64
low       851264 non-null float64
high      851264 non-null float64
volume    851264 non-null float64
dtypes: float64(5), object(2)
memory usage: 45.5+ MB


### Drop columns `date` and  `symbol`

In [9]:
data = data.drop('date', axis=1)
data = data.drop('symbol', axis=1)

In [10]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set

In [11]:
data_1000 = data.loc[0:999]

In [12]:
data_1000.shape

(1000, 5)

### Divide the data into train and test sets

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
Y=data_1000['close']
X=data_1000.drop('close',axis=1)

In [15]:
trainX, testX,trainY,testY = train_test_split(X, Y, test_size=0.3, random_state=100)

In [16]:
trainY.shape

(700,)

## Building the graph in tensorflow

1.Define input data placeholders

In [17]:
#Input features
x = tf.placeholder(shape=[None,4],dtype=tf.float32, name='x-input')

#Normalize the data
x_n = tf.layers.batch_normalization(x, training=True)

#Actual Prices
y_ = tf.placeholder(shape=[None],dtype=tf.float32, name='y-input')

Instructions for updating:
Use keras.layers.batch_normalization instead.
Instructions for updating:
Colocations handled automatically by placer.


2.Define Weights and Bias

In [18]:
W = tf.Variable(tf.zeros(shape=[4,1]), name="Weights")
b = tf.Variable(tf.zeros(shape=[1]), name="Bias")

3.Prediction

In [19]:
y = tf.add(tf.matmul(x_n,W),b,name='output')

4.Loss (Cost) Function [Mean square error]

In [20]:
loss = tf.reduce_mean(tf.square(y-y_),name='Loss')

5.GradientDescent Optimizer to minimize Loss [GradientDescentOptimizer]

In [21]:
train_op = tf.train.GradientDescentOptimizer(0.01).minimize(loss)

In [22]:
y_.shape

TensorShape([Dimension(None)])

## Execute the Graph for 100 epochs and observe the loss

In [23]:
#Lets start graph Execution
sess = tf.Session()

# variables need to be initialized before we can use them
sess.run(tf.global_variables_initializer())

#how many times data need to be shown to model
training_epochs = 100

In [24]:
for epoch in range(training_epochs):
            
    #Calculate train_op and loss
    _, train_loss = sess.run([train_op, loss],feed_dict={x:trainX, 
                                                        y_:trainY})
    
    if epoch % 10 == 0:
        print ('Training loss at step: ', epoch, ' is ', train_loss)

Training loss at step:  0  is  8324.746
Training loss at step:  10  is  6848.4756
Training loss at step:  20  is  5766.7646
Training loss at step:  30  is  3884.3655
Training loss at step:  40  is  3883.7715
Training loss at step:  50  is  3883.6292
Training loss at step:  60  is  3883.553
Training loss at step:  70  is  3883.5068
Training loss at step:  80  is  3883.477
Training loss at step:  90  is  3883.4568


### Get the shapes and values of W and b

Hint: Use sess.run(W) to get W.

In [25]:
sess.run(W)

array([[-2.2273   ],
       [-1.3949951],
       [ 5.861472 ],
       [ 0.6248641]], dtype=float32)

In [26]:
sess.run(b)

array([24.586452], dtype=float32)

### Find the Absolute mean square loss difference between training and testing loss.

In [27]:
for epoch in range(training_epochs):
            
    #Calculate train_op and loss
    _, test_loss = sess.run([train_op, loss],feed_dict={x:testX, 
                                                        y_:testY})
    
    if epoch % 10 == 0:
        print ('Testing loss at step: ', epoch, ' is ', test_loss)

Testing loss at step:  0  is  2280.653
Testing loss at step:  10  is  2280.4458
Testing loss at step:  20  is  2280.4373
Testing loss at step:  30  is  2280.4307
Testing loss at step:  40  is  2280.4255
Testing loss at step:  50  is  2280.4216
Testing loss at step:  60  is  2280.4182
Testing loss at step:  70  is  2280.4153
Testing loss at step:  80  is  2280.4128
Testing loss at step:  90  is  2280.4111


In [28]:
test_loss

2280.4092

In [29]:
train_loss

3883.4436

In [30]:
train_loss-test_loss

1603.0344

### Linear Classification using Keras

### Building the simple Neural Network in Keras with one neuron in the dense hidden layer.
#### Use Mean square error as loss function and sgd as optimizer

In [31]:
#Initialize Sequential Graph (model)
model = tf.keras.models.Sequential()

#Normalize input data
model.add(tf.keras.layers.BatchNormalization(input_shape=(4,)))

#Add Dense layer for prediction - Keras declares weights and bias automatically
model.add(tf.keras.layers.Dense(1))

#set up learning rate 
sgd = tf.keras.optimizers.SGD(lr = 0.003)

#Compile the model - add Loss and Gradient Descent optimizer
model.compile(optimizer= sgd, loss='mse')

Instructions for updating:
Use tf.cast instead.


In [32]:
trainX.shape

(700, 4)

In [33]:
trainY.shape

(700,)

In [34]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_v1 (Batc (None, 4)                 16        
_________________________________________________________________
dense (Dense)                (None, 1)                 5         
Total params: 21
Trainable params: 13
Non-trainable params: 8
_________________________________________________________________


### Execute the model

In [35]:
model.fit(trainX,trainY,validation_data=(testX,testY), epochs=100)

Train on 700 samples, validate on 300 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/100
700/700 [==============================] - 1s 1ms/sample - loss: 3431.9278 - val_loss: 4253.3102
Epoch 2/100
700/700 [==============================] - 0s 66us/sample - loss: 1592.0368 - val_loss: 31712.6188
Epoch 3/100
700/700 [==============================] - 0s 67us/sample - loss: 1532.4615 - val_loss: 646.9730
Epoch 4/100
700/700 [==============================] - 0s 81us/sample - loss: 1596.0315 - val_loss: 1717.3655
Epoch 5/100
700/700 [==============================] - 0s 80us/sample - loss: 2036.9064 - val_loss: 176.1865
Epoch 6/100
700/700 [==============================] - 0s 58us/sample - loss: 1727.1143 - val_loss: 1702.8438
Epoch 7/100
700/700 [==============================] - 0s 68us/sample - loss: 1157.8057 - val_loss: 126.0936
Epoch 8/100
700/700 [==============================] - 0s 58us/sample - loss: 1204.4019 - val_loss: 2535.2415
Epoch 9/100
700/700 [========

Epoch 73/100
700/700 [==============================] - 0s 77us/sample - loss: 1380.9548 - val_loss: 337.8775
Epoch 74/100
700/700 [==============================] - 0s 56us/sample - loss: 1572.8415 - val_loss: 383.2285
Epoch 75/100
700/700 [==============================] - 0s 68us/sample - loss: 1381.4048 - val_loss: 2076.2938
Epoch 76/100
700/700 [==============================] - 0s 67us/sample - loss: 1623.3534 - val_loss: 352.2999
Epoch 77/100
700/700 [==============================] - 0s 64us/sample - loss: 1394.1092 - val_loss: 581.1548
Epoch 78/100
700/700 [==============================] - 0s 60us/sample - loss: 1439.0746 - val_loss: 1580.2829
Epoch 79/100
700/700 [==============================] - 0s 68us/sample - loss: 1591.1500 - val_loss: 203.0406
Epoch 80/100
700/700 [==============================] - 0s 67us/sample - loss: 1355.0051 - val_loss: 5.1135
Epoch 81/100
700/700 [==============================] - 0s 66us/sample - loss: 1370.2667 - val_loss: 320.3751
Epoch 82/1

In [36]:
model.get_weights()

[array([8.697211  , 3.4075046 , 5.608842  , 0.31690016], dtype=float32),
 array([ -3.4717348,  -1.3380382,  -2.208862 , -10.577698 ], dtype=float32),
 array([6.6615608e+01, 6.5928551e+01, 6.7292595e+01, 5.1737830e+06],
       dtype=float32),
 array([3.7999189e+03, 3.7365183e+03, 3.8376284e+03, 1.4554003e+14],
       dtype=float32),
 array([[ 7.5817966],
        [ 2.9757588],
        [ 4.8966217],
        [-9.008403 ]], dtype=float32),
 array([15.988707], dtype=float32)]

In [51]:
sess.close()

### Classification using Keras 

### Load the given Iris data using pandas (Iris.csv)

In [38]:
Source = pd.read_csv('Iris-2.csv')

In [39]:
Source.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 6 columns):
Id               150 non-null int64
SepalLengthCm    150 non-null float64
SepalWidthCm     150 non-null float64
PetalLengthCm    150 non-null float64
PetalWidthCm     150 non-null float64
Species          150 non-null object
dtypes: float64(4), int64(1), object(1)
memory usage: 7.1+ KB


### Splitting the data into feature set and target set

In [40]:
Source = Source.drop('Id', axis=1)

In [41]:
Y = Source['Species']
X = Source.drop('Species', axis = 1)

In [42]:
Y.head()

0    Iris-setosa
1    Iris-setosa
2    Iris-setosa
3    Iris-setosa
4    Iris-setosa
Name: Species, dtype: object

### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [43]:
Y_dummies = pd.DataFrame(pd.get_dummies(Y, sparse=True).values)

In [45]:
Y_dummies.shape

(150, 3)

### Divide the dataset into Training and test (70:30)

In [46]:
trainX, testX,trainY,testY = train_test_split(X, Y_dummies, test_size=0.3, random_state=100)

### Model
Build the model with following layers: <br>
1. First dense layer with 10 neurons with input shape 4 (according to the feature set) <br>
2. Second Dense layer with 8 neurons <br>
3. Output layer with 3 neurons with softmax activation (output layer, 3 neurons as we have 3 classes) <br>
4. Use SGD and categorical_crossentropy loss 

In [47]:
#Initialize the sequential model
model_iris = tf.keras.models.Sequential()

#Normalize the data
model_iris.add(tf.keras.layers.BatchNormalization())

#Add Dense layer which provides 10 outputs after applying softmax
model_iris.add(tf.keras.layers.Dense(10, activation='sigmoid', input_dim=4))

#Add Dense layer which provides 10 outputs after applying softmax
model_iris.add(tf.keras.layers.Dense(8, activation='sigmoid'))

#sgd = tf.keras.optimizers.SGD(lr = 0.003)

#Add Dense layer which provides 10 outputs after applying softmax
model_iris.add(tf.keras.layers.Dense(3, activation='softmax'))

In [48]:
#Complile the model
model_iris.compile(optimizer = 'sgd', loss = 'categorical_crossentropy', metrics = ['accuracy'])

### Fitting the model and predicting 

In [49]:
model_iris.fit(trainX.values,trainY.values,validation_data=(testX,testY), epochs=1000)

Train on 105 samples, validate on 45 samples
Epoch 1/1000
105/105 [==============================] - 1s 5ms/sample - loss: 1.4325 - acc: 0.3238 - val_loss: 1.2264 - val_acc: 0.3556
Epoch 2/1000
105/105 [==============================] - 0s 161us/sample - loss: 1.4139 - acc: 0.3238 - val_loss: 1.2181 - val_acc: 0.3556
Epoch 3/1000
105/105 [==============================] - 0s 190us/sample - loss: 1.3980 - acc: 0.3238 - val_loss: 1.2066 - val_acc: 0.3556
Epoch 4/1000
105/105 [==============================] - 0s 180us/sample - loss: 1.3782 - acc: 0.3238 - val_loss: 1.1970 - val_acc: 0.3556
Epoch 5/1000
105/105 [==============================] - 0s 171us/sample - loss: 1.3616 - acc: 0.3238 - val_loss: 1.1880 - val_acc: 0.3556
Epoch 6/1000
105/105 [==============================] - 0s 142us/sample - loss: 1.3439 - acc: 0.3048 - val_loss: 1.1796 - val_acc: 0.3556
Epoch 7/1000
105/105 [==============================] - 0s 161us/sample - loss: 1.3288 - acc: 0.2762 - val_loss: 1.1734 - val_acc

Epoch 59/1000
105/105 [==============================] - 0s 152us/sample - loss: 1.0911 - acc: 0.3524 - val_loss: 1.0923 - val_acc: 0.3556
Epoch 60/1000
105/105 [==============================] - 0s 152us/sample - loss: 1.0905 - acc: 0.3429 - val_loss: 1.0921 - val_acc: 0.3778
Epoch 61/1000
105/105 [==============================] - 0s 152us/sample - loss: 1.0900 - acc: 0.3714 - val_loss: 1.0920 - val_acc: 0.5111
Epoch 62/1000
105/105 [==============================] - 0s 161us/sample - loss: 1.0890 - acc: 0.4571 - val_loss: 1.0919 - val_acc: 0.6000
Epoch 63/1000
105/105 [==============================] - 0s 162us/sample - loss: 1.0872 - acc: 0.5714 - val_loss: 1.0920 - val_acc: 0.6000
Epoch 64/1000
105/105 [==============================] - 0s 171us/sample - loss: 1.0868 - acc: 0.5619 - val_loss: 1.0909 - val_acc: 0.6000
Epoch 65/1000
105/105 [==============================] - 0s 161us/sample - loss: 1.0854 - acc: 0.6000 - val_loss: 1.0907 - val_acc: 0.6000
Epoch 66/1000
105/105 [====

Epoch 118/1000
105/105 [==============================] - 0s 162us/sample - loss: 1.0487 - acc: 0.4476 - val_loss: 1.0668 - val_acc: 0.3333
Epoch 119/1000
105/105 [==============================] - 0s 152us/sample - loss: 1.0486 - acc: 0.4381 - val_loss: 1.0657 - val_acc: 0.3333
Epoch 120/1000
105/105 [==============================] - 0s 114us/sample - loss: 1.0473 - acc: 0.4286 - val_loss: 1.0658 - val_acc: 0.3333
Epoch 121/1000
105/105 [==============================] - 0s 152us/sample - loss: 1.0470 - acc: 0.4190 - val_loss: 1.0643 - val_acc: 0.3556
Epoch 122/1000
105/105 [==============================] - 0s 133us/sample - loss: 1.0488 - acc: 0.4857 - val_loss: 1.0646 - val_acc: 0.3333
Epoch 123/1000
105/105 [==============================] - 0s 152us/sample - loss: 1.0447 - acc: 0.4286 - val_loss: 1.0639 - val_acc: 0.3333
Epoch 124/1000
105/105 [==============================] - 0s 152us/sample - loss: 1.0435 - acc: 0.4286 - val_loss: 1.0629 - val_acc: 0.3333
Epoch 125/1000
105/1

105/105 [==============================] - 0s 171us/sample - loss: 1.0060 - acc: 0.6571 - val_loss: 1.0145 - val_acc: 0.5778
Epoch 177/1000
105/105 [==============================] - 0s 199us/sample - loss: 1.0041 - acc: 0.6762 - val_loss: 1.0129 - val_acc: 0.5778
Epoch 178/1000
105/105 [==============================] - 0s 152us/sample - loss: 1.0041 - acc: 0.6857 - val_loss: 1.0119 - val_acc: 0.5778
Epoch 179/1000
105/105 [==============================] - 0s 161us/sample - loss: 1.0004 - acc: 0.6952 - val_loss: 1.0109 - val_acc: 0.5778
Epoch 180/1000
105/105 [==============================] - 0s 152us/sample - loss: 1.0002 - acc: 0.6952 - val_loss: 1.0092 - val_acc: 0.5778
Epoch 181/1000
105/105 [==============================] - 0s 161us/sample - loss: 0.9996 - acc: 0.6952 - val_loss: 1.0096 - val_acc: 0.5778
Epoch 182/1000
105/105 [==============================] - 0s 190us/sample - loss: 1.0004 - acc: 0.6952 - val_loss: 1.0104 - val_acc: 0.5778
Epoch 183/1000
105/105 [===========

105/105 [==============================] - 0s 171us/sample - loss: 0.9475 - acc: 0.6952 - val_loss: 0.9471 - val_acc: 0.6000
Epoch 235/1000
105/105 [==============================] - 0s 161us/sample - loss: 0.9454 - acc: 0.6952 - val_loss: 0.9456 - val_acc: 0.6000
Epoch 236/1000
105/105 [==============================] - 0s 142us/sample - loss: 0.9500 - acc: 0.6952 - val_loss: 0.9439 - val_acc: 0.6000
Epoch 237/1000
105/105 [==============================] - 0s 152us/sample - loss: 0.9428 - acc: 0.6952 - val_loss: 0.9419 - val_acc: 0.6000
Epoch 238/1000
105/105 [==============================] - 0s 152us/sample - loss: 0.9432 - acc: 0.6952 - val_loss: 0.9396 - val_acc: 0.6000
Epoch 239/1000
105/105 [==============================] - 0s 161us/sample - loss: 0.9453 - acc: 0.6857 - val_loss: 0.9397 - val_acc: 0.6000
Epoch 240/1000
105/105 [==============================] - 0s 142us/sample - loss: 0.9393 - acc: 0.6952 - val_loss: 0.9393 - val_acc: 0.6000
Epoch 241/1000
105/105 [===========

...

105/105 [==============================] - 0s 152us/sample - loss: 0.5067 - acc: 0.9048 - val_loss: 0.4812 - val_acc: 0.9333
Epoch 757/1000
105/105 [==============================] - 0s 161us/sample - loss: 0.4975 - acc: 0.9143 - val_loss: 0.4813 - val_acc: 0.9333
Epoch 758/1000
105/105 [==============================] - 0s 123us/sample - loss: 0.5028 - acc: 0.9238 - val_loss: 0.4806 - val_acc: 0.9333
Epoch 759/1000
105/105 [==============================] - 0s 142us/sample - loss: 0.5075 - acc: 0.9048 - val_loss: 0.4805 - val_acc: 0.9333
Epoch 760/1000
105/105 [==============================] - 0s 171us/sample - loss: 0.5031 - acc: 0.8762 - val_loss: 0.4803 - val_acc: 0.9333
Epoch 761/1000
105/105 [==============================] - 0s 152us/sample - loss: 0.5011 - acc: 0.9048 - val_loss: 0.4800 - val_acc: 0.9333
Epoch 762/1000
105/105 [==============================] - 0s 180us/sample - loss: 0.5086 - acc: 0.7905 - val_loss: 0.4792 - val_acc: 0.9333
Epoch 763/1000
105/105 [===========

105/105 [==============================] - 0s 161us/sample - loss: 0.4927 - acc: 0.9429 - val_loss: 0.4668 - val_acc: 0.9556
Epoch 814/1000
105/105 [==============================] - 0s 152us/sample - loss: 0.4970 - acc: 0.8286 - val_loss: 0.4661 - val_acc: 0.9556
Epoch 815/1000
105/105 [==============================] - 0s 161us/sample - loss: 0.4842 - acc: 0.9429 - val_loss: 0.4665 - val_acc: 0.9333
Epoch 816/1000
105/105 [==============================] - 0s 171us/sample - loss: 0.4895 - acc: 0.8952 - val_loss: 0.4658 - val_acc: 0.9556
Epoch 817/1000
105/105 [==============================] - ETA: 0s - loss: 0.4748 - acc: 0.937 - 0s 228us/sample - loss: 0.4869 - acc: 0.9143 - val_loss: 0.4657 - val_acc: 0.9556
Epoch 818/1000
105/105 [==============================] - 0s 180us/sample - loss: 0.4834 - acc: 0.9333 - val_loss: 0.4656 - val_acc: 0.9556
Epoch 819/1000
105/105 [==============================] - 0s 161us/sample - loss: 0.4865 - acc: 0.9143 - val_loss: 0.4646 - val_acc: 0.95

105/105 [==============================] - 0s 161us/sample - loss: 0.4715 - acc: 0.8952 - val_loss: 0.4608 - val_acc: 0.9333
Epoch 872/1000
105/105 [==============================] - 0s 152us/sample - loss: 0.4760 - acc: 0.8952 - val_loss: 0.4605 - val_acc: 0.9556
Epoch 873/1000
105/105 [==============================] - 0s 133us/sample - loss: 0.4753 - acc: 0.9143 - val_loss: 0.4606 - val_acc: 0.9333
Epoch 874/1000
105/105 [==============================] - 0s 143us/sample - loss: 0.4733 - acc: 0.9048 - val_loss: 0.4601 - val_acc: 0.9333
Epoch 875/1000
105/105 [==============================] - 0s 171us/sample - loss: 0.4771 - acc: 0.9048 - val_loss: 0.4593 - val_acc: 0.9333
Epoch 876/1000
105/105 [==============================] - 0s 142us/sample - loss: 0.4785 - acc: 0.8762 - val_loss: 0.4597 - val_acc: 0.9333
Epoch 877/1000
105/105 [==============================] - 0s 171us/sample - loss: 0.4714 - acc: 0.8952 - val_loss: 0.4599 - val_acc: 0.9333
Epoch 878/1000
105/105 [===========

105/105 [==============================] - 0s 190us/sample - loss: 0.4656 - acc: 0.9238 - val_loss: 0.4502 - val_acc: 0.9333
Epoch 930/1000
105/105 [==============================] - 0s 152us/sample - loss: 0.4651 - acc: 0.9333 - val_loss: 0.4500 - val_acc: 0.9333
Epoch 931/1000
105/105 [==============================] - 0s 171us/sample - loss: 0.4690 - acc: 0.8952 - val_loss: 0.4505 - val_acc: 0.9333
Epoch 932/1000
105/105 [==============================] - 0s 123us/sample - loss: 0.4663 - acc: 0.8952 - val_loss: 0.4505 - val_acc: 0.9333
Epoch 933/1000
105/105 [==============================] - 0s 142us/sample - loss: 0.4748 - acc: 0.8571 - val_loss: 0.4494 - val_acc: 0.9333
Epoch 934/1000
105/105 [==============================] - 0s 152us/sample - loss: 0.4711 - acc: 0.8762 - val_loss: 0.4486 - val_acc: 0.9333
Epoch 935/1000
105/105 [==============================] - 0s 162us/sample - loss: 0.4671 - acc: 0.9143 - val_loss: 0.4483 - val_acc: 0.9333
Epoch 936/1000
105/105 [===========

105/105 [==============================] - 0s 161us/sample - loss: 0.4525 - acc: 0.9524 - val_loss: 0.4395 - val_acc: 0.9556
Epoch 988/1000
105/105 [==============================] - ETA: 0s - loss: 0.4578 - acc: 0.843 - 0s 180us/sample - loss: 0.4571 - acc: 0.8762 - val_loss: 0.4389 - val_acc: 0.9556
Epoch 989/1000
105/105 [==============================] - 0s 152us/sample - loss: 0.4554 - acc: 0.9143 - val_loss: 0.4382 - val_acc: 0.9556
Epoch 990/1000
105/105 [==============================] - 0s 152us/sample - loss: 0.4605 - acc: 0.8667 - val_loss: 0.4375 - val_acc: 0.9556
Epoch 991/1000
105/105 [==============================] - 0s 152us/sample - loss: 0.4562 - acc: 0.9143 - val_loss: 0.4373 - val_acc: 0.9556
Epoch 992/1000
105/105 [==============================] - 0s 133us/sample - loss: 0.4535 - acc: 0.9429 - val_loss: 0.4376 - val_acc: 0.9556
Epoch 993/1000
105/105 [==============================] - 0s 152us/sample - loss: 0.4545 - acc: 0.8857 - val_loss: 0.4368 - val_acc: 0.95

### Report Accuracy of the predicted values

In [50]:
model_iris.evaluate(testX,testY)

45/45 [==============================] - 0s 133us/sample - loss: 0.4385 - acc: 0.9333


[0.4384958333439297, 0.93333334]